# 🍜🍝 Fusion Cuisine Project: Complete Interactive Pipeline

## Overview
This notebook provides a comprehensive interface to conduct the entire Fusion Cuisine project including:
- **Data Generation & Analysis**
- **Model Training & Evaluation** 
- **Fusion Recipe Creation**
- **Real-time Log Monitoring**
- **Performance Visualization**
- **Interactive Recipe Explorer**

---

## 🔧 Setup and Configuration

In [ ]:
import sys
import os
import time
import json
import threading
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Jupyter widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Markdown

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Import project modules
from app.config import config
from app.utils import setup_logging, create_log_monitor, format_time

# Setup
setup_logging("INFO")
plt.style.use('default')
sns.set_palette("husl")

print("🚀 Fusion Cuisine Project Setup Complete!")
print(f"📁 Project root: {project_root}")
print(f"⚙️ Configuration: {config.config_path}")
print(f"🔧 Offline mode: {config.offline_mode}")
print(f"🎲 Random seed: {config.random_seed}")


## 📊 Log Monitoring Setup

In [ ]:
# Create log monitor for real-time tracking
log_monitor = create_log_monitor()

# Global variables for tracking
pipeline_stats = {
    'start_time': None,
    'step_times': {},
    'current_step': None,
    'total_steps': 6
}

def start_pipeline_tracking():
    """Start pipeline tracking and monitoring."""
    pipeline_stats['start_time'] = time.time()
    log_monitor.start_monitoring()
    print("🔍 Log monitoring started")
    print(f"📊 Monitoring log file: {log_monitor.log_file}")

def track_step(step_name):
    """Track the start of a pipeline step."""
    if pipeline_stats['current_step']:
        # End previous step
        prev_step = pipeline_stats['current_step']
        pipeline_stats['step_times'][prev_step] = time.time() - pipeline_stats['step_times'][prev_step]
    
    pipeline_stats['current_step'] = step_name
    pipeline_stats['step_times'][step_name] = time.time()
    
    print(f"\n🔄 Starting: {step_name}")
    print(f"⏰ Time: {datetime.now().strftime('%H:%M:%S')}")

def show_monitoring_stats():
    """Display current monitoring statistics."""
    stats = log_monitor.get_stats()
    
    print("\n📈 CURRENT MONITORING STATS")
    print("=" * 40)
    print(f"📝 Log lines processed: {stats['total_lines']}")
    print(f"ℹ️ Info messages: {stats['info_count']}")
    print(f"⚠️ Warnings: {stats['warning_count']}")
    print(f"❌ Errors: {stats['error_count']}")
    print(f"🚨 Critical: {stats['critical_count']}")
    
    if stats.get('runtime_seconds'):
        print(f"⏱️ Monitoring runtime: {format_time(stats['runtime_seconds'])}")
        print(f"📊 Processing rate: {stats.get('lines_per_second', 0):.2f} lines/sec")

print("✅ Log monitoring system ready")


## 🎮 Interactive Pipeline Control

In [ ]:
# Create interactive widgets for pipeline control
output_area = widgets.Output()

# Step selection
step_selector = widgets.SelectMultiple(
    options=[
        ('1. Data Generation', 'data'),
        ('2. Data Preprocessing', 'preprocess'),
        ('3. Train Encoder', 'encoder'),
        ('4. Train PalateNet', 'palatenet'),
        ('5. Generate Fusion Recipes', 'fusion'),
        ('6. Evaluate Results', 'evaluate')
    ],
    value=['data', 'preprocess', 'encoder', 'palatenet', 'fusion', 'evaluate'],
    description='Steps to run:',
    disabled=False,
    layout=widgets.Layout(height='150px')
)

# Configuration options
offline_mode = widgets.Checkbox(
    value=config.offline_mode,
    description='Offline Mode',
    disabled=False
)

num_recipes = widgets.IntSlider(
    value=config.target_recipes_per_cuisine,
    min=100,
    max=10000,
    step=100,
    description='Recipes per cuisine:',
    style={'description_width': 'initial'}
)

alpha_values = widgets.Text(
    value=', '.join(map(str, config.fusion_alphas)),
    description='Alpha values:',
    placeholder='0.3, 0.5, 0.7'
)

# Control buttons
run_button = widgets.Button(
    description='🚀 Run Pipeline',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

monitor_button = widgets.Button(
    description='📊 Show Monitoring',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

clear_button = widgets.Button(
    description='🧹 Clear Output',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

# Layout
config_box = widgets.VBox([
    widgets.HTML("<h3>⚙️ Configuration</h3>"),
    offline_mode,
    num_recipes,
    alpha_values
])

steps_box = widgets.VBox([
    widgets.HTML("<h3>📋 Pipeline Steps</h3>"),
    step_selector
])

controls_box = widgets.VBox([
    widgets.HTML("<h3>🎮 Controls</h3>"),
    widgets.HBox([run_button, monitor_button, clear_button])
])

main_controls = widgets.HBox([config_box, steps_box, controls_box])

display(main_controls)
display(output_area)


In [ ]:
# Button event handlers
def run_pipeline(b):
    """Run the selected pipeline steps."""
    with output_area:
        clear_output()
        
        # Start monitoring
        start_pipeline_tracking()
        
        print("🚀 FUSION CUISINE PIPELINE STARTING")
        print("=" * 50)
        print(f"📅 Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🔧 Offline mode: {offline_mode.value}")
        print(f"📊 Recipes per cuisine: {num_recipes.value}")
        print(f"🎯 Alpha values: {alpha_values.value}")
        print(f"📋 Steps to run: {', '.join(step_selector.value)}")
        
        try:
            # Update config if needed
            if offline_mode.value != config.offline_mode:
                os.environ['OFFLINE_MODE'] = '1' if offline_mode.value else '0'
            
            # Change to project directory
            original_dir = os.getcwd()
            os.chdir(project_root)
            
            # Run selected steps
            step_scripts = {
                'data': 'scripts/01_fetch_data.py',
                'preprocess': 'scripts/02_preprocess.py',
                'encoder': 'scripts/train_encoder.py',
                'palatenet': 'scripts/train_palatenet.py',
                'fusion': 'scripts/generate_fusion.py',
                'evaluate': 'scripts/evaluate.py'
            }
            
            step_names = {
                'data': 'Data Generation',
                'preprocess': 'Data Preprocessing', 
                'encoder': 'Encoder Training',
                'palatenet': 'PalateNet Training',
                'fusion': 'Fusion Generation',
                'evaluate': 'Evaluation'
            }
            
            for step in step_selector.value:
                if step in step_scripts:
                    track_step(step_names[step])
                    
                    # Run the script
                    script_path = step_scripts[step]
                    print(f"📜 Executing: {script_path}")
                    
                    exec(open(script_path).read(), globals())
                    
                    step_time = pipeline_stats['step_times'][step_names[step]]
                    print(f"✅ {step_names[step]} completed in {format_time(time.time() - step_time)}")
            
            # Restore original directory
            os.chdir(original_dir)
            
            # Final step tracking
            if pipeline_stats['current_step']:
                prev_step = pipeline_stats['current_step']
                pipeline_stats['step_times'][prev_step] = time.time() - pipeline_stats['step_times'][prev_step]
            
            total_time = time.time() - pipeline_stats['start_time']
            
            print("\n🎉 PIPELINE COMPLETED SUCCESSFULLY!")
            print("=" * 50)
            print(f"⏱️ Total time: {format_time(total_time)}")
            print(f"📅 Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            # Show step breakdown
            print("\n📊 Step Breakdown:")
            for step_name, step_time in pipeline_stats['step_times'].items():
                if isinstance(step_time, (int, float)):
                    percentage = (step_time / total_time) * 100
                    print(f"   {step_name}: {format_time(step_time)} ({percentage:.1f}%)")
            
        except Exception as e:
            print(f"\n❌ Pipeline failed with error: {e}")
            import traceback
            traceback.print_exc()
        
        finally:
            # Stop monitoring
            log_monitor.stop_monitoring()
            print("\n🔍 Log monitoring stopped")

def show_monitoring(b):
    """Display monitoring statistics."""
    with output_area:
        show_monitoring_stats()
        
        # Show recent errors if any
        recent_errors = log_monitor.get_recent_errors(5)
        if recent_errors:
            print("\n🔴 Recent Errors:")
            for error in recent_errors:
                print(f"   • {error}")
        else:
            print("\n✅ No recent errors")

def clear_output_area(b):
    """Clear the output area."""
    with output_area:
        clear_output()
        print("🧹 Output cleared")

# Attach event handlers
run_button.on_click(run_pipeline)
monitor_button.on_click(show_monitoring)
clear_button.on_click(clear_output_area)

print("✅ Interactive controls ready!")
print("👆 Use the controls above to run the pipeline")


## 📊 Real-time Results Visualization

In [ ]:
def create_pipeline_dashboard():
    """Create an interactive dashboard for pipeline results."""
    
    # Check for available data
    data_files = {
        'recipes': config.data_dir / "recipes.csv",
        'ratings': config.data_dir / "ratings.csv", 
        'evaluation': config.outputs_dir / "evaluation_metrics.json",
        'fusion_recipes': config.outputs_dir / "recipes" / "fusion_recipes.json"
    }
    
    available_data = {k: v.exists() for k, v in data_files.items()}
    
    print("📊 DATA AVAILABILITY:")
    for name, available in available_data.items():
        status = "✅" if available else "❌"
        print(f"   {status} {name}")
    
    if not any(available_data.values()):
        print("\n⚠️ No data available yet. Run the pipeline first!")
        return
    
    # Create dashboard sections
    dashboard_tabs = widgets.Tab()
    tab_contents = []
    tab_titles = []
    
    # Data Overview Tab
    if available_data['recipes']:
        data_output = widgets.Output()
        with data_output:
            try:
                df = pd.read_csv(data_files['recipes'])
                
                print(f"📊 DATASET OVERVIEW")
                print(f"   Total recipes: {len(df):,}")
                print(f"   Cuisines: {', '.join(df['cuisine'].unique())}")
                print(f"   Average rating: {df['rating'].mean():.2f}")
                print(f"   Rating range: {df['rating'].min():.1f} - {df['rating'].max():.1f}")
                
                # Create visualizations
                fig = make_subplots(
                    rows=2, cols=2,
                    subplot_titles=['Recipes by Cuisine', 'Rating Distribution', 
                                  'Ingredients per Recipe', 'Rating by Cuisine'],
                    specs=[[{'type': 'bar'}, {'type': 'histogram'}],
                           [{'type': 'histogram'}, {'type': 'box'}]]
                )
                
                # Cuisine distribution
                cuisine_counts = df['cuisine'].value_counts()
                fig.add_trace(
                    go.Bar(x=cuisine_counts.index, y=cuisine_counts.values, name='Recipes'),
                    row=1, col=1
                )
                
                # Rating distribution
                fig.add_trace(
                    go.Histogram(x=df['rating'], nbinsx=20, name='Ratings'),
                    row=1, col=2
                )
                
                # Ingredients per recipe
                if 'num_ingredients' in df.columns:
                    fig.add_trace(
                        go.Histogram(x=df['num_ingredients'], nbinsx=15, name='Ingredients'),
                        row=2, col=1
                    )
                
                # Rating by cuisine
                for cuisine in df['cuisine'].unique():
                    cuisine_ratings = df[df['cuisine'] == cuisine]['rating']
                    fig.add_trace(
                        go.Box(y=cuisine_ratings, name=cuisine),
                        row=2, col=2
                    )
                
                fig.update_layout(height=600, showlegend=False, title_text="Data Analysis Dashboard")
                fig.show()
                
            except Exception as e:
                print(f"Error creating data overview: {e}")
        
        tab_contents.append(data_output)
        tab_titles.append("Data Overview")
    
    # Evaluation Results Tab
    if available_data['evaluation']:
        eval_output = widgets.Output()
        with eval_output:
            try:
                with open(data_files['evaluation']) as f:
                    metrics = json.load(f)
                
                print("📈 EVALUATION RESULTS")
                print("=" * 30)
                
                # Rating prediction metrics
                if 'rating_prediction' in metrics:
                    rating_metrics = metrics['rating_prediction']
                    print("\n🎯 Rating Prediction Performance:")
                    for metric, value in rating_metrics.items():
                        if isinstance(value, (int, float)):
                            print(f"   {metric}: {value:.4f}")
                        else:
                            print(f"   {metric}: {value}")
                
                # Ingredient overlap metrics
                if 'ingredient_overlap' in metrics:
                    overlap_metrics = metrics['ingredient_overlap']
                    print("\n🥘 Ingredient Overlap Analysis:")
                    for metric, value in overlap_metrics.items():
                        if isinstance(value, dict):
                            print(f"   {metric}:")
                            for k, v in value.items():
                                print(f"      {k}: {v:.4f}")
                        elif isinstance(value, (int, float)):
                            print(f"   {metric}: {value:.4f}")
                
                # Create evaluation visualization
                if 'rating_prediction' in metrics and 'ingredient_overlap' in metrics:
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=['Model Performance', 'Overlap by Alpha']
                    )
                    
                    # Performance metrics
                    perf_metrics = metrics['rating_prediction']
                    metric_names = ['spearman_correlation', 'r2_score']
                    metric_values = [perf_metrics.get(m, 0) for m in metric_names]
                    
                    fig.add_trace(
                        go.Bar(x=['Spearman Corr', 'R² Score'], y=metric_values),
                        row=1, col=1
                    )
                    
                    # Alpha overlap
                    alpha_overlaps = metrics['ingredient_overlap'].get('alpha_specific_overlaps', {})
                    if alpha_overlaps:
                        alphas = [float(k.split('_')[1]) for k in alpha_overlaps.keys()]
                        overlaps = list(alpha_overlaps.values())
                        
                        fig.add_trace(
                            go.Scatter(x=alphas, y=overlaps, mode='lines+markers'),
                            row=1, col=2
                        )
                    
                    fig.update_layout(height=400, title_text="Evaluation Results")
                    fig.show()
                
            except Exception as e:
                print(f"Error displaying evaluation results: {e}")
        
        tab_contents.append(eval_output)
        tab_titles.append("Evaluation")
    
    # Fusion Recipes Tab
    if available_data['fusion_recipes']:
        fusion_output = widgets.Output()
        with fusion_output:
            try:
                with open(data_files['fusion_recipes']) as f:
                    fusion_recipes = json.load(f)
                
                print("🍽️ FUSION RECIPES GENERATED")
                print("=" * 35)
                
                for alpha_key, recipe_data in fusion_recipes.items():
                    alpha = recipe_data['alpha']
                    recipe = recipe_data['recipe']
                    
                    print(f"\n🎯 Alpha = {alpha:.1f} (Japanese: {alpha:.1f}, Italian: {1-alpha:.1f})")
                    print("-" * 60)
                    print(recipe[:400] + "..." if len(recipe) > 400 else recipe)
                    print("-" * 60)
                
            except Exception as e:
                print(f"Error displaying fusion recipes: {e}")
        
        tab_contents.append(fusion_output)
        tab_titles.append("Fusion Recipes")
    
    # Setup tabs
    if tab_contents:
        dashboard_tabs.children = tab_contents
        for i, title in enumerate(tab_titles):
            dashboard_tabs.set_title(i, title)
        
        display(dashboard_tabs)
    else:
        print("⚠️ No results available to display")

# Create dashboard button
dashboard_button = widgets.Button(
    description='📊 Show Dashboard',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

dashboard_output = widgets.Output()

def show_dashboard(b):
    with dashboard_output:
        clear_output()
        create_pipeline_dashboard()

dashboard_button.on_click(show_dashboard)

display(dashboard_button)
display(dashboard_output)


## 🔍 Interactive Recipe Explorer

In [ ]:
def create_recipe_explorer():
    """Create an interactive recipe exploration interface."""
    
    explorer_output = widgets.Output()
    
    # Recipe type selector
    recipe_type = widgets.Dropdown(
        options=[('Original Recipes', 'original'), ('Fusion Recipes', 'fusion')],
        value='original',
        description='Recipe Type:'
    )
    
    # Cuisine selector (for original recipes)
    cuisine_selector = widgets.Dropdown(
        options=[('Japanese', 'japanese'), ('Italian', 'italian'), ('All', 'all')],
        value='all',
        description='Cuisine:'
    )
    
    # Alpha selector (for fusion recipes)
    alpha_selector = widgets.Dropdown(
        options=[('Alpha 0.3', 0.3), ('Alpha 0.5', 0.5), ('Alpha 0.7', 0.7)],
        value=0.5,
        description='Alpha Value:'
    )
    
    # Recipe index slider
    recipe_index = widgets.IntSlider(
        value=0,
        min=0,
        max=10,
        description='Recipe #:'
    )
    
    # Random recipe button
    random_button = widgets.Button(
        description='🎲 Random Recipe',
        button_style='info'
    )
    
    def update_recipe_display(*args):
        with explorer_output:
            clear_output()
            
            try:
                if recipe_type.value == 'original':
                    # Display original recipes
                    recipes_path = config.data_dir / "recipes.csv"
                    if not recipes_path.exists():
                        print("❌ Original recipes not found. Run data generation first.")
                        return
                    
                    df = pd.read_csv(recipes_path)
                    
                    if cuisine_selector.value != 'all':
                        df = df[df['cuisine'] == cuisine_selector.value]
                    
                    if len(df) == 0:
                        print("❌ No recipes found for selected criteria.")
                        return
                    
                    # Update slider max
                    recipe_index.max = len(df) - 1
                    
                    if recipe_index.value >= len(df):
                        recipe_index.value = 0
                    
                    recipe = df.iloc[recipe_index.value]
                    
                    print(f"🍽️ {recipe['cuisine'].upper()} RECIPE #{recipe_index.value + 1}")
                    print("=" * 50)
                    print(f"📝 Title: {recipe['title']}")
                    print(f"⭐ Rating: {recipe['rating']:.1f}/5.0")
                    if 'num_ingredients' in recipe:
                        print(f"🥘 Ingredients: {recipe['num_ingredients']}")
                    print(f"\n📋 Ingredients:")
                    print(recipe['ingredients'][:300] + "..." if len(recipe['ingredients']) > 300 else recipe['ingredients'])
                    print(f"\n👨‍🍳 Instructions:")
                    print(recipe['instructions'][:500] + "..." if len(recipe['instructions']) > 500 else recipe['instructions'])
                    
                else:
                    # Display fusion recipes
                    fusion_path = config.outputs_dir / "recipes" / "fusion_recipes.json"
                    if not fusion_path.exists():
                        print("❌ Fusion recipes not found. Run fusion generation first.")
                        return
                    
                    with open(fusion_path) as f:
                        fusion_recipes = json.load(f)
                    
                    # Find recipe with matching alpha
                    selected_recipe = None
                    for key, recipe_data in fusion_recipes.items():
                        if abs(recipe_data['alpha'] - alpha_selector.value) < 0.01:
                            selected_recipe = recipe_data
                            break
                    
                    if not selected_recipe:
                        print(f"❌ No fusion recipe found for alpha = {alpha_selector.value}")
                        return
                    
                    alpha = selected_recipe['alpha']
                    recipe_text = selected_recipe['recipe']
                    
                    print(f"🌏 FUSION RECIPE (α = {alpha})")
                    print("=" * 50)
                    print(f"🍣 Japanese influence: {alpha:.1f} ({alpha*100:.0f}%)")
                    print(f"🍝 Italian influence: {1-alpha:.1f} ({(1-alpha)*100:.0f}%)")
                    print(f"\n📝 Recipe:")
                    print("-" * 50)
                    print(recipe_text)
                    print("-" * 50)
                
            except Exception as e:
                print(f"❌ Error displaying recipe: {e}")
    
    def random_recipe(b):
        import random
        if recipe_type.value == 'original':
            try:
                recipes_path = config.data_dir / "recipes.csv"
                if recipes_path.exists():
                    df = pd.read_csv(recipes_path)
                    if cuisine_selector.value != 'all':
                        df = df[df['cuisine'] == cuisine_selector.value]
                    if len(df) > 0:
                        recipe_index.value = random.randint(0, len(df) - 1)
            except:
                pass
        else:
            # Random alpha for fusion recipes
            alpha_options = [0.3, 0.5, 0.7]
            alpha_selector.value = random.choice(alpha_options)
    
    # Attach event handlers
    recipe_type.observe(update_recipe_display, 'value')
    cuisine_selector.observe(update_recipe_display, 'value')
    alpha_selector.observe(update_recipe_display, 'value')
    recipe_index.observe(update_recipe_display, 'value')
    random_button.on_click(random_recipe)
    
    # Layout
    controls = widgets.HBox([
        widgets.VBox([recipe_type, cuisine_selector]),
        widgets.VBox([alpha_selector, recipe_index]),
        widgets.VBox([random_button])
    ])
    
    display(widgets.HTML("<h3>🔍 Recipe Explorer</h3>"))
    display(controls)
    display(explorer_output)
    
    # Initial display
    update_recipe_display()

create_recipe_explorer()


## 📈 Performance Analytics

In [ ]:
def analyze_performance():
    """Analyze and visualize system performance."""
    
    print("📈 PERFORMANCE ANALYTICS")
    print("=" * 40)
    
    # Pipeline timing analysis
    if pipeline_stats['step_times']:
        print("\n⏱️ Pipeline Timing:")
        total_time = sum(t for t in pipeline_stats['step_times'].values() if isinstance(t, (int, float)))
        
        for step, time_taken in pipeline_stats['step_times'].items():
            if isinstance(time_taken, (int, float)):
                percentage = (time_taken / total_time) * 100 if total_time > 0 else 0
                print(f"   {step}: {format_time(time_taken)} ({percentage:.1f}%)")
        
        print(f"   Total: {format_time(total_time)}")
        
        # Create timing visualization
        step_names = list(pipeline_stats['step_times'].keys())
        step_times = [t for t in pipeline_stats['step_times'].values() if isinstance(t, (int, float))]
        
        if step_times:
            fig = go.Figure(data=[
                go.Pie(labels=step_names, values=step_times, hole=0.3)
            ])
            fig.update_layout(
                title="Pipeline Time Distribution",
                annotations=[dict(text='Pipeline', x=0.5, y=0.5, font_size=20, showarrow=False)]
            )
            fig.show()
    
    # Log monitoring analysis
    log_stats = log_monitor.get_stats()
    if log_stats['total_lines'] > 0:
        print("\n📊 Log Analysis:")
        print(f"   Total log entries: {log_stats['total_lines']:,}")
        print(f"   Info messages: {log_stats['info_count']:,}")
        print(f"   Warnings: {log_stats['warning_count']:,}")
        print(f"   Errors: {log_stats['error_count']:,}")
        print(f"   Critical: {log_stats['critical_count']:,}")
        
        if log_stats.get('runtime_seconds'):
            print(f"   Processing rate: {log_stats.get('lines_per_second', 0):.2f} lines/sec")
            print(f"   Error rate: {log_stats.get('errors_per_minute', 0):.2f} errors/min")
            print(f"   Warning rate: {log_stats.get('warnings_per_minute', 0):.2f} warnings/min")
        
        # Log level distribution
        log_levels = ['info_count', 'warning_count', 'error_count', 'critical_count']
        log_counts = [log_stats[level] for level in log_levels]
        log_labels = ['Info', 'Warning', 'Error', 'Critical']
        
        if sum(log_counts) > 0:
            fig = go.Figure(data=[
                go.Bar(x=log_labels, y=log_counts, 
                      marker_color=['blue', 'orange', 'red', 'darkred'])
            ])
            fig.update_layout(title="Log Message Distribution", yaxis_title="Count")
            fig.show()
    
    # System resource analysis (if available)
    try:
        import psutil
        
        print("\n💻 System Resources:")
        print(f"   CPU usage: {psutil.cpu_percent()}%")
        print(f"   Memory usage: {psutil.virtual_memory().percent}%")
        print(f"   Available memory: {psutil.virtual_memory().available / (1024**3):.1f} GB")
        
        # Disk usage for project directory
        disk_usage = psutil.disk_usage(str(project_root))
        print(f"   Disk usage: {(disk_usage.used / disk_usage.total) * 100:.1f}%")
        print(f"   Free disk space: {disk_usage.free / (1024**3):.1f} GB")
        
    except ImportError:
        print("\n💻 System resource monitoring not available (psutil not installed)")
    
    # File size analysis
    print("\n📁 Generated Files:")
    
    file_sizes = {}
    
    # Data files
    data_files = [
        "recipes.csv", "ratings.csv", "flavor_graph.json",
        "text_embeddings.npy", "ingredient_embeddings.npy"
    ]
    
    for filename in data_files:
        filepath = config.data_dir / filename
        if filepath.exists():
            size_mb = filepath.stat().st_size / (1024**2)
            file_sizes[f"data/{filename}"] = size_mb
            print(f"   📊 {filename}: {size_mb:.1f} MB")
    
    # Model files
    model_files = ["encoder_best.pt", "palatenet_best.pt"]
    
    for filename in model_files:
        filepath = config.models_dir / filename
        if filepath.exists():
            size_mb = filepath.stat().st_size / (1024**2)
            file_sizes[f"models/{filename}"] = size_mb
            print(f"   🧠 {filename}: {size_mb:.1f} MB")
    
    # Output files
    output_files = ["evaluation_metrics.json", "recipes/fusion_recipes.json"]
    
    for filename in output_files:
        filepath = config.outputs_dir / filename
        if filepath.exists():
            size_kb = filepath.stat().st_size / 1024
            file_sizes[f"outputs/{filename}"] = size_kb / 1024  # Convert to MB
            print(f"   📋 {filename}: {size_kb:.1f} KB")
    
    # Create file size visualization
    if file_sizes:
        fig = go.Figure(data=[
            go.Bar(x=list(file_sizes.keys()), y=list(file_sizes.values()))
        ])
        fig.update_layout(
            title="File Sizes (MB)",
            xaxis_title="Files",
            yaxis_title="Size (MB)",
            xaxis_tickangle=-45
        )
        fig.show()
    
    total_size = sum(file_sizes.values())
    print(f"\n📦 Total project size: {total_size:.1f} MB")

# Performance analysis button
perf_button = widgets.Button(
    description='📈 Analyze Performance',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

perf_output = widgets.Output()

def show_performance(b):
    with perf_output:
        clear_output()
        analyze_performance()

perf_button.on_click(show_performance)

display(perf_button)
display(perf_output)


## 🛠️ Advanced Tools

In [ ]:
# Export results function
def export_results():
    """Export all results to a comprehensive report."""
    export_dir = project_root / "exports"
    export_dir.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_file = export_dir / f"fusion_cuisine_report_{timestamp}.html"
    
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Fusion Cuisine Project Report</title>
        <style>
            body {{ font-family: Arial, sans-serif; margin: 40px; }}
            h1 {{ color: #2E8B57; }}
            h2 {{ color: #4682B4; }}
            .metrics {{ background-color: #f0f8ff; padding: 15px; border-radius: 5px; }}
            .recipe {{ background-color: #fff8dc; padding: 15px; margin: 10px 0; border-radius: 5px; }}
        </style>
    </head>
    <body>
        <h1>🍜🍝 Fusion Cuisine Project Report</h1>
        <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>
        
        <h2>📊 Project Configuration</h2>
        <div class="metrics">
            <p><strong>Offline Mode:</strong> {config.offline_mode}</p>
            <p><strong>Target Cuisines:</strong> {', '.join(config.target_cuisines)}</p>
            <p><strong>Recipes per Cuisine:</strong> {config.target_recipes_per_cuisine:,}</p>
            <p><strong>Alpha Values:</strong> {', '.join(map(str, config.fusion_alphas))}</p>
            <p><strong>Random Seed:</strong> {config.random_seed}</p>
        </div>
    """
    
    # Add pipeline timing if available
    if pipeline_stats['step_times']:
        html_content += """
        <h2>⏱️ Pipeline Performance</h2>
        <div class="metrics">
        """
        
        total_time = sum(t for t in pipeline_stats['step_times'].values() if isinstance(t, (int, float)))
        
        for step, time_taken in pipeline_stats['step_times'].items():
            if isinstance(time_taken, (int, float)):
                percentage = (time_taken / total_time) * 100 if total_time > 0 else 0
                html_content += f"<p><strong>{step}:</strong> {format_time(time_taken)} ({percentage:.1f}%)</p>"
        
        html_content += f"<p><strong>Total Time:</strong> {format_time(total_time)}</p>"
        html_content += "</div>"
    
    # Add evaluation metrics if available
    try:
        eval_path = config.outputs_dir / "evaluation_metrics.json"
        if eval_path.exists():
            with open(eval_path) as f:
                metrics = json.load(f)
            
            html_content += """
            <h2>📈 Evaluation Results</h2>
            <div class="metrics">
            """
            
            if 'rating_prediction' in metrics:
                rating_metrics = metrics['rating_prediction']
                html_content += "<h3>Rating Prediction Performance</h3>"
                for metric, value in rating_metrics.items():
                    if isinstance(value, (int, float)):
                        html_content += f"<p><strong>{metric.replace('_', ' ').title()}:</strong> {value:.4f}</p>"
            
            if 'ingredient_overlap' in metrics:
                overlap_metrics = metrics['ingredient_overlap']
                html_content += "<h3>Ingredient Overlap Analysis</h3>"
                html_content += f"<p><strong>Average Overlap:</strong> {overlap_metrics.get('avg_ingredient_overlap', 0):.4f}</p>"
            
            html_content += "</div>"
    except:
        pass
    
    # Add fusion recipes if available
    try:
        fusion_path = config.outputs_dir / "recipes" / "fusion_recipes.json"
        if fusion_path.exists():
            with open(fusion_path) as f:
                fusion_recipes = json.load(f)
            
            html_content += """
            <h2>🍽️ Generated Fusion Recipes</h2>
            """
            
            for alpha_key, recipe_data in fusion_recipes.items():
                alpha = recipe_data['alpha']
                recipe = recipe_data['recipe']
                
                html_content += f"""
                <div class="recipe">
                    <h3>Alpha = {alpha:.1f} (Japanese: {alpha:.1f}, Italian: {1-alpha:.1f})</h3>
                    <p>{recipe.replace('\n', '<br>')}</p>
                </div>
                """
    except:
        pass
    
    html_content += """
        <h2>📁 Project Files</h2>
        <div class="metrics">
            <p><strong>Data Directory:</strong> {}</p>
            <p><strong>Models Directory:</strong> {}</p>
            <p><strong>Outputs Directory:</strong> {}</p>
        </div>
        
        <footer>
            <hr>
            <p><em>Generated by Fusion Cuisine Project Notebook</em></p>
        </footer>
    </body>
    </html>
    """.format(config.data_dir, config.models_dir, config.outputs_dir)
    
    with open(report_file, 'w') as f:
        f.write(html_content)
    
    print(f"📄 Report exported to: {report_file}")
    return report_file

# Clean up function
def cleanup_project():
    """Clean up project files."""
    import shutil
    
    cleanup_dirs = [
        config.data_dir,
        config.models_dir,
        config.outputs_dir / "recipes",
        config.outputs_dir / "images"
    ]
    
    total_size = 0
    
    for dir_path in cleanup_dirs:
        if dir_path.exists():
            # Calculate size before deletion
            for file_path in dir_path.rglob('*'):
                if file_path.is_file():
                    total_size += file_path.stat().st_size
            
            shutil.rmtree(dir_path)
            dir_path.mkdir(exist_ok=True)
    
    print(f"🧹 Cleaned up {total_size / (1024**2):.1f} MB of project files")
    print("✅ Project reset and ready for new run")

# Tool buttons
export_button = widgets.Button(
    description='📄 Export Report',
    button_style='success',
    layout=widgets.Layout(width='150px')
)

cleanup_button = widgets.Button(
    description='🧹 Clean Project',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

tools_output = widgets.Output()

def export_report(b):
    with tools_output:
        clear_output()
        report_file = export_results()
        print(f"✅ Report generated successfully!")
        print(f"📁 Location: {report_file}")

def cleanup_files(b):
    with tools_output:
        clear_output()
        confirm = input("⚠️ This will delete all generated files. Type 'yes' to confirm: ")
        if confirm.lower() == 'yes':
            cleanup_project()
        else:
            print("❌ Cleanup cancelled")

export_button.on_click(export_report)
cleanup_button.on_click(cleanup_files)

tools_box = widgets.HBox([export_button, cleanup_button])

display(widgets.HTML("<h3>🛠️ Advanced Tools</h3>"))
display(tools_box)
display(tools_output)


---

## 🎉 Congratulations!

You now have a **complete interactive interface** to conduct the Fusion Cuisine project!

### 🚀 Quick Start Guide:
1. **Configure** your settings using the dropdown menus above
2. **Select steps** to run (or run all for complete pipeline)
3. **Click "🚀 Run Pipeline"** to start the process
4. **Monitor progress** using the "📊 Show Monitoring" button
5. **Explore results** with the Dashboard and Recipe Explorer
6. **Analyze performance** to understand system behavior
7. **Export report** for documentation and sharing

### 📊 What This Notebook Provides:
- ✅ **Interactive Pipeline Control** - Run any combination of steps
- ✅ **Real-time Log Monitoring** - Track errors and performance
- ✅ **Dynamic Visualizations** - Plotly charts and analysis
- ✅ **Recipe Explorer** - Browse original and fusion recipes
- ✅ **Performance Analytics** - Timing and resource usage
- ✅ **Export Capabilities** - Generate HTML reports
- ✅ **Project Management** - Clean up and reset functionality

### 🔥 Advanced Features:
- **Configurable Parameters** - Adjust recipes count, alpha values, offline mode
- **Step-by-step Execution** - Run individual pipeline components
- **Error Detection** - Automatic monitoring with alerts
- **Interactive Widgets** - Full Jupyter widget integration
- **Comprehensive Logging** - Track all system activities

**Happy experimenting with fusion cuisine! 👨‍🍳👩‍🍳🍜🍝**